# Build Container

In [1]:
%%sh 

# The name of our algorithm
algorithm_name=bert-inf1-serving
account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR

docker build --no-cache -t ${algorithm_name} -f Dockerfile.sm.neuron-rtd .
docker tag ${algorithm_name} ${fullname}
docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  1.927MB
Step 1/22 : FROM amazonlinux:2
 ---> ba2cc467a2bc
Step 2/22 : MAINTAINER Chaitanya Hazarey <chazarey@amazon.com>
 ---> Running in a6580b07f9c7
Removing intermediate container a6580b07f9c7
 ---> 1235849a8769
Step 3/22 : RUN echo $'[neuron] \nname=Neuron YUM Repository \nbaseurl=https://yum.repos.neuron.amazonaws.com \nenabled=1' > /etc/yum.repos.d/neuron.repo
 ---> Running in 2e517c0b2be5
Removing intermediate container 2e517c0b2be5
 ---> cb2877d1c528
Step 4/22 : RUN rpm --import https://yum.repos.neuron.amazonaws.com/GPG-PUB-KEY-AMAZON-AWS-NEURON.PUB
 ---> Running in 2bf2b1845c03
Removing intermediate container 2bf2b1845c03
 ---> 9c842341a89d
Step 5/22 : RUN yum install -y wget
 ---> Running in ea24b7e26232
Loaded plugins: ovl, priorities
Resolving Dependencies
--> Running transaction check
---> Package wget.x86_64 0:1.14-18.amzn2.1 will be installed
--> Processing Dependency: libidn.so.11(LIBIDN_1.0)(64bit) for package: w

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

COPY failed: stat /var/lib/docker/tmp/docker-builder162144040/bert-base-uncased-saved: no such file or directory
